In [1]:
import requests
from bs4 import BeautifulSoup
# import streamlit as st
from transformers import pipeline
import re
# import pymongo
# from pymongo import MongoClient



# specify the URL of the news website to scrape
url = "https://www.theverge.com/"

# send a GET request to the website URL and store the response
response = requests.get(url)

# parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# find all the news article links on the website
article_links = soup.find_all("a", class_="group-hover:shadow-underline-franklin")

l = []
links = []

# loop through each article link and scrape the article text
for link in article_links:
    # extract the URL of the article
    article_url = link["href"]
    
    # send a GET request to the article URL and store the response
    article_response = requests.get(url + article_url)
    links.append(url + article_url)

#     # parse the HTML content of the article using BeautifulSoup
    article_soup = BeautifulSoup(article_response.content, "html.parser")
    
#     # find the main text content of the article and print it
    article_text = article_soup.find("div", class_="clearfix").get_text()
    l.append(article_text)


In [2]:
def get_first_thousand_words(text):
    words = re.findall(r'\b\w+\b|[^\w\s]', text)
    return ' '.join(words[:800])

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = []
for i in l:
    i = get_first_thousand_words(i)

    summaries.append(summarizer(i, max_length=250, min_length=100))

summariesText = [i[0]["summary_text"] for i in summaries]

In [3]:

import requests
from bs4 import BeautifulSoup
# import streamlit as st
from transformers import pipeline
import re
# import pymongo
# from pymongo import MongoClient



# specify the URL of the news website to scrape
url = "https://www.theverge.com/"

# send a GET request to the website URL and store the response
response = requests.get(url)

# parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# find all the news article links on the website
article_links = soup.find_all("a", class_="group-hover:shadow-underline-franklin")

l = []
links = []

# loop through each article link and scrape the article text
for link in article_links:
    # extract the URL of the article
    article_url = link["href"]
    
    # send a GET request to the article URL and store the response
    article_response = requests.get(url + article_url)
    links.append(url + article_url)

#     # parse the HTML content of the article using BeautifulSoup
    article_soup = BeautifulSoup(article_response.content, "html.parser")
    
#     # find the main text content of the article and print it
    article_text = article_soup.find("div", class_="clearfix").get_text()
    l.append(article_text)

def get_first_thousand_words(text):
    words = re.findall(r'\b\w+\b|[^\w\s]', text)
    return ' '.join(words[:800])

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = []
for i in l:
    i = get_first_thousand_words(i)

    summaries.append(summarizer(i, max_length=250, min_length=100))

summariesText = [i[0]["summary_text"] for i in summaries]

In [4]:
links

['https://www.theverge.com//2023/4/21/23692368/humane-ted-talk-imran-chaudhri-wearable-screenless-device-voice-commands-projected-screen',
 'https://www.theverge.com//2023/4/21/23692425/apple-journaling-app-jurassic-wwdc-2023-sherlocking-ios-17',
 'https://www.theverge.com//2023/4/21/23692254/whatsapp-keep-in-chat-disappearing-messages',
 'https://www.theverge.com//2023/4/20/23689570/activitypub-protocol-standard-social-network',
 'https://www.theverge.com//2023/4/20/23691831/twitter-blue-verified-celebrity-lebron-james-stephen-king']

In [5]:
summariesText

['Humane, the startup founded by ex - Apple employees Imran Chaudhri and Bethany Bongiorno, has given a first live demo of its new device. The device is a wearable gadget with a projected display and AI - powered features intended to act as a personal assistant. It is expected to be released to the public on April 22nd and is not expected to require a smartphone or other device to pair with it. In the presentation, ChaudHri uses the device to identify a chocolate bar and advise him whether or not to eat it based on his dietary requirements.',
 'Apple could offer its own journaling app with the next update to iOS, according to a new report. The software, codenamed Jurassic, will attempt to tap into the apparent mental and physical benefits of logging your thoughts and activities on a regular basis. The app could be announced as early as Apple’s Worldwide Developers Conference in June as a feature for iOS 17. It could bring Apple into direct competition with third - party journaling apps

In [6]:
import sqlite3




# Connect to the database
conn = sqlite3.connect('NewsSummarizer.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Delete all existing rows from the 'verge' table
sql_command = "DELETE FROM verge"
cursor.execute(sql_command)

# SQL command to insert sample data into the 'verge' table
sql_command = "INSERT INTO verge (summary, link) VALUES (?, ?)"

# Execute the SQL command for each row of sample data
for i in range(len(summariesText)):
    row = (str(summariesText[i])[2:-3], links[i],)
    cursor.execute(sql_command, row)

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()


In [ ]:
summariesText

In [7]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('NewsSummarizer.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# SQL command to select table names
sql_command = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the SQL command
cursor.execute(sql_command)

# Fetch all the table names as a list of tuples
table_names = cursor.fetchall()

# Print the table names
print("Available tables:")
for table in table_names:
    print(table[0])

# Close the database connection
conn.close()


Available tables:
verge


In [ ]:
# Connect to the database
conn = sqlite3.connect('NewsSummarizer.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Select all rows from the 'verge' table
sql_command = "SELECT * FROM verge"
cursor.execute(sql_command)

summaries_final = []
links_final = []
# Fetch all rows and print them
rows = cursor.fetchall()
for row in rows:
    summaries_final.append(row[0])
    links_final.append(row[1])    

# Close the database connection
conn.close()


In [ ]:
summaries_final

In [ ]:
summaries = []
links = []
for i in fromDB:
    summaries.append(i[0])
    links.append(i[1])

In [ ]:
links

In [ ]:
import dropbox

# Enter your access token here
ACCESS_TOKEN = 'sl.BbVyAS1TBPadIhcDKEl86-drmfkyVwUu54C5OoZ4TxNFEX-lzhs2HZoGI6dsaL1UXz0L2wWBrMBfYv7x-5nTeMKleoWzaKzbdBqxX1XNYkQGkwtHN0h2z8nsl-GWxn-ro7KiIkE'


# Write the lists to a file
with open('data.txt', 'w') as f:
    f.write('\n'.join(summariesText))
    f.write('\n')
    f.write('\n'.join(links))

# Create a Dropbox object
dbx = dropbox.Dropbox(ACCESS_TOKEN)

# Upload the file to Dropbox
with open('data.txt', 'rb') as f:
    dbx.files_upload(f.read(), '/Apps/NewsSummarizer/data.txt', mode=dropbox.files.WriteMode.overwrite)
